In [ ]:
# Setup

# Install packages
!pip install smartapi-python
!pip install logzero
!pip install pyotp
!pip install websocket-client
!pip install ijson
!pip install mplfinance
!pip install pandas plotly

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/

# Set time zone to IST
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!date


In [ ]:

# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from datetime import time, timedelta, datetime
import ijson
import json
import mplfinance as mpf
import time
import plotly.graph_objects as go
import requests
import re
import pandas as pd
from talib.abstract import *

In [ ]:
def data_cleaning(file_name):

    df = pd.read_csv(file_name)

    # Use regex to extract the date
    match = re.search(r'_(\d{8})\.csv$', file_name)
    if match:
        date_str = match.group(1)  # Extract the date part

        # Convert the extracted date string to a datetime object
        date_obj = datetime.strptime(date_str, "%m%d%Y")

        # Format the date to YYYY-MM-DD
        formatted_date = date_obj.strftime("%Y-%m-%d")
    else:
        formatted_date = None  # Handle case where date is not found

    df = df.drop(columns=['Volume', 'Open Interest'])
    df['Time'] = df['Time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
    df = df[
        df['Ticker'].str.startswith('NIFTY') &
        (df['Ticker'].str.endswith('PE.NFO') | df['Ticker'].str.endswith('CE.NFO'))
    ]
    # Define the target date
    target_date = pd.to_datetime(formatted_date)

    # Function to extract date from ticker
    def extract_date(ticker):
        match = re.search(r'(\d{2})([A-Z]{3})(\d{2})', ticker)
        if match:
            day = int(match.group(1))
            month_str = match.group(2)
            year = int(match.group(3)) + 2000  # Assuming year is in 2000s
            month = pd.to_datetime(month_str, format='%b').month
            return pd.Timestamp(year=year, month=month, day=day)
        return None

    # Apply the function to extract dates
    df['Extracted_Date'] = df['Ticker'].apply(extract_date)

    # Calculate the absolute difference from the target date
    df['Date_Difference'] = (df['Extracted_Date'] - target_date).abs()

    # Find the minimum difference
    min_difference = df['Date_Difference'].min()

    # Filter the DataFrame to keep only the closest date rows
    df = df[df['Date_Difference'] == min_difference]

    df = df.drop(columns=['Extracted_Date', 'Date_Difference', 'Date'])

    return df


df = data_cleaning("/content/GFDLNFO_BACKADJUSTED_01042024.csv")
df2 = data_cleaning("/content/GFDLNFO_BACKADJUSTED_04042024.csv")

In [ ]:


def add_time_to_string(time_str, m):
    # Parse the time string into a datetime object
    time_obj = datetime.strptime(time_str, "%H:%M:%S")

    # Add the specified hours and minutes
    new_time_obj = time_obj + timedelta(minutes=m)

    # Format the new time back to a string
    new_time_str = new_time_obj.strftime("%H:%M:%S")

    return new_time_str

def find_nearest_options(df, price_input, curr_time):
    # Convert curr_time to match the format in the DataFrame (HH:MM)
    curr_time_formatted = pd.to_datetime(curr_time).strftime('%H:%M:%S')

    # Filter rows where the time matches curr_time (ignoring seconds)
    df_filtered = df[df['Time'] == curr_time_formatted]

    # Extract hour and minute from x
    curr_time = pd.to_datetime(curr_time, format='%H:%M:%S')

    # Filter rows where the hour and minute match
    df_filtered = df[(df['Time'].apply(lambda x: x.hour) == curr_time.hour) &
                  (df['Time'].apply(lambda x: x.minute) == curr_time.minute)]

    # Find difference
    df_filtered['Closest'] = abs(df_filtered['Close'] - price_input)

    # Separate CE and PE options (using endswith)
    ce_options = df_filtered[df_filtered['Ticker'].str.endswith('CE.NFO')]
    pe_options = df_filtered[df_filtered['Ticker'].str.endswith('PE.NFO')]

    # Find the first CE and PE with the smallest Close >= price_input
    nearest_ce = ce_options.loc[ce_options['Closest'].idxmin()] if not ce_options.empty else None
    nearest_pe = pe_options.loc[pe_options['Closest'].idxmin()] if not pe_options.empty else None

    return nearest_ce, nearest_pe

def sl_and_reentry(df, ticker, price, time):

    # Initialize variables
    stop_loss_price = 60  # Stop loss threshold
    reentry_price = 40    # Re-entry threshold
    profit_loss = 0       # Track profit/loss
    active = True         # Track if the position is active
    curr_time = time      # Start tracking from the given time
    profit_p = []
    curr_price = price
    re_entry = 0

    # Filter rows by name
    df_filtered = df[(df['Ticker'] == ticker)]

    while True:

        # Extract hour and minute from x
        df_filtered = df[(df['Ticker'] == ticker)]
        curr_time = pd.to_datetime(curr_time, format='%H:%M:%S')

        # Assuming curr_time is a datetime.time object
        df_filtered = df[
            (
                (df['Time'].apply(lambda x: x.hour) > curr_time.hour)
            ) |
            (
                (df['Time'].apply(lambda x: x.hour) == curr_time.hour) &
                (df['Time'].apply(lambda x: x.minute) > curr_time.minute)
            )
        ]

        print("---------CHECK 1-----------------")
        print(df_filtered)
        print(curr_time)

        if active:

            # Filter by price
            df_filtered_price = df_filtered[(df_filtered['High'] >= stop_loss_price)]

            # No SL hit
            if (len(df_filtered_price) == 0):
                df_filtered = df_filtered.sort_values(by='Time')
                print("No SL")
                print(df_filtered)
                profit_p.append((- df_filtered.iloc[-1]['Close'] + curr_price) / 200000 * 100 * 75)
                print("----------------------------------------")
                print("Sell", curr_price)
                print("Buy", df_filtered.iloc[-1]['Close'])
                print("----------------------------------------")
                return profit_p
            else:
                # Sort by Time and get the first row
                print("-----------------SL--------------------")
                result = df_filtered_price.sort_values(by='Time')
                print(result)
                first_r = result.iloc[0]
                curr_time = first_r['Time']
                profit_p.append((-first_r['High'] + curr_price) / 200000 * 100 * 75)
                print("----------------------------------------")
                print("Sell", curr_price)
                print("Buy", first_r['High'])
                print("----------------------------------------")
                active = False

        else:

            # Filter by price
            df_filtered_price = df_filtered[(df_filtered['Low'] <= reentry_price)]

            # No reentry
            if (len(df_filtered_price) == 0):
                print("---------------------------------------No reentry--------------------------------")
                return profit_p
            else:
                print("---------------------------------------Reentry--------------------------------")
                if re_entry == 2:
                    return profit_p
                # Sort by Time and get the first row
                re_entry += 1
                result = df_filtered_price.sort_values(by='Time')
                print(result)
                first_r = result.iloc[0]
                curr_time = first_r['Time']
                active = True
                curr_price = first_r['Low']


    return None

def backtest(df):

    curr_time = "09:30:00"
    active = set()
    profit = 0
    profit_p = []
    price_input = 40
    profit_track = []



    while curr_time <= "10:00:00":

        curr = find_nearest_options(df, price_input, curr_time)

        profit_p.extend(sl_and_reentry(df, curr[0]['Ticker'], curr[0]['Close'], curr_time))
        profit_p.extend(sl_and_reentry(df, curr[1]['Ticker'], curr[1]['Close'], curr_time))

        curr_time = add_time_to_string(curr_time, 30)


    # Initialize variables
    total_sum = sum(profit_p)  # Sum of all values
    num_win_days = sum(1 for value in profit_p if value > 0)  # Count of positive values
    num_loss_days = sum(1 for value in profit_p if value < 0)  # Count of negative values

    # Print results
    print(profit_p)
    print("Total Profit %:", total_sum)
    print("Number of wins:", num_win_days)
    print("Number of losses:", num_loss_days)



backtest(df2)

df2


In [ ]:
dfc = pd.read_csv("/content/GFDLNFO_BACKADJUSTED_04042024.csv")
dfc = dfc.drop(columns=['Volume', 'Open Interest'])
dfc['Time'] = dfc['Time'].apply(lambda x: datetime.strptime(x, "%H:%M:%S").time())
dfc = dfc[
    dfc['Ticker'] == "NIFTY04APR2422400PE.NFO"]
dfc = dfc.sort_values(by='Time')
dfc